In [175]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 

In [176]:
file_path = "TushyCartonisationAnalysis_recreate.xlsx"
excel_sheets = pd.ExcelFile(file_path).sheet_names

In [177]:
invoice_df = pd.read_excel(file_path, sheet_name='Raw Baja Invoices')

invoice_df['Charge'] = invoice_df['Charge'].str.upper()
invoice_df_drop_columns = ['Reference','Entry Date', 'Entry Number', 'Customs Value Currency']
invoice_df.drop(columns=invoice_df_drop_columns,axis=1,inplace=True)
invoice_df['id'] = range(1, len(invoice_df) + 1)

invoice_df_data_types = {"Order Number":"str"}
invoice_df = invoice_df.astype(invoice_df_data_types)

In [178]:
charge_df = pd.read_excel(file_path, sheet_name='Charge Map')
charge_df['Charge'] = charge_df['Charge'].str.upper()
charge_map_df = charge_df[["Charge","Map"]]

In [179]:
invoice_charge_df = invoice_df.merge(charge_map_df, on='Charge', how='inner')

In [180]:
#missing Charge "Return To Sender - Web Request" and map "Returns"
invoice_charge_df["Map"].unique()

array(['Freight', 'Fuel', 'Residential', 'Surcharge', 'Freight Adj',
       'Fuel Adj', 'Surcharge Adj', 'Duties/Tax'], dtype=object)

In [181]:
invoice_Flatten = invoice_charge_df.copy()
unique_maps = invoice_Flatten['Map'].unique()
for map_value in unique_maps:
    invoice_Flatten[map_value] = invoice_Flatten.apply(
        lambda row: row['Bill Amount'] if row['Map'] == map_value else 0, axis=1
    )

In [182]:
invoice_Flatten.head()

,Invoice,Invoice Date,Order ID,Order Number,Profile,Carrier,Transaction Date,Tracking Number,Service Type,Charge,Package Quantity,Packaging Type,Entered Weight (LB),Billed Weight (LB),Dim Length,Dim Width,Dim Height,Zone,Origin State,Origin Country,Receiver Name,Receiver Company,Receiver Address Line 1,Receiver Address Line 2,Receiver City,Receiver State,Receiver Zip,Receiver Country,Customs Value,Duty & Taxes,Bill Amount,id,Map,Freight,Fuel,Residential,Surcharge,Freight Adj,Fuel Adj,Surcharge Adj,Duties/Tax
0,33477,2025-03-20,576899052,#1442861,default,UPS,45699,1ZAC8375YW34252402,UPS SurePost - 1 lb or Greater,TRANSPORTATION,1.0,PKG,4.1,5.0,19.0,9.0,5.0,7,TX,US,Dana Wallace,"TUSHY, Inc",10813 NW 30TH ST STE 115,NaN,DORAL,FL,331922147,US,0.0,0.0,9.80,1,Freight,9.80,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,33477,2025-03-20,576899052,#1442861,default,UPS,45699,1ZAC8375YW34252402,UPS SurePost - 1 lb or Greater,FUEL SURCHARGE,1.0,PKG,4.1,5.0,19.0,9.0,5.0,7,TX,US,Dana Wallace,"TUSHY, Inc",10813 NW 30TH ST STE 115,NaN,DORAL,FL,331922147,US,0.0,0.0,1.23,2,Fuel,0.00,1.23,0.0,0.0,0.0,0.0,0.0,0.0
2,33477,2025-03-20,581683521,#1446209,default,UPS,45712,1ZAC8375YW14943702,UPS SurePost - 1 lb or Greater,TRANSPORTATION,1.0,PKG,2.1,3.0,18.0,10.0,4.0,7,TX,US,NaN,Jacob Vosper,22 DORLAND AVE,NaN,POUGHKEEPSIE,NY,126036404,US,0.0,0.0,8.90,3,Freight,8.90,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,33477,2025-03-20,581683521,#1446209,default,UPS,45712,1ZAC8375YW14943702,UPS SurePost - 1 lb or Greater,FUEL SURCHARGE,1.0,PKG,2.1,3.0,18.0,10.0,4.0,7,TX,US,NaN,Jacob Vosper,22 DORLAND AVE,NaN,POUGHKEEPSIE,NY,126036404,US,0.0,0.0,1.12,4,Fuel,0.00,1.12,0.0,0.0,0.0,0.0,0.0,0.0
4,33477,2025-03-20,582604555,#1446902,default,UPS,45714,1ZAC83750301909861,Ground Residential,TRANSPORTATION,1.0,PKG,0.1,18.0,23.0,21.0,8.0,7,TX,US,NaN,Alisha Sare,277 SANTA ROSA AVE,PUSH 3 ON THE HILAVATOR OR TAKE,SAUSALITO,CA,949652036,US,0.0,0.0,16.86,5,Freight,16.86,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
invoice_charge_df.columns

Index(['Invoice', 'Invoice Date', 'Order ID', 'Order Number', 'Profile',
       'Carrier', 'Transaction Date', 'Tracking Number', 'Service Type',
       'Charge', 'Package Quantity', 'Packaging Type', 'Entered Weight (LB)',
       'Billed Weight (LB)', 'Dim Length', 'Dim Width', 'Dim Height', 'Zone',
       'Origin State', 'Origin Country', 'Receiver Name', 'Receiver Company',
       'Receiver Address Line 1', 'Receiver Address Line 2', 'Receiver City',
       'Receiver State', 'Receiver Zip', 'Receiver Country', 'Customs Value',
       'Duty & Taxes', 'Bill Amount', 'id', 'Map'],
      dtype='object')

In [184]:
invoice_Flatten.columns

Index(['Invoice', 'Invoice Date', 'Order ID', 'Order Number', 'Profile',
       'Carrier', 'Transaction Date', 'Tracking Number', 'Service Type',
       'Charge', 'Package Quantity', 'Packaging Type', 'Entered Weight (LB)',
       'Billed Weight (LB)', 'Dim Length', 'Dim Width', 'Dim Height', 'Zone',
       'Origin State', 'Origin Country', 'Receiver Name', 'Receiver Company',
       'Receiver Address Line 1', 'Receiver Address Line 2', 'Receiver City',
       'Receiver State', 'Receiver Zip', 'Receiver Country', 'Customs Value',
       'Duty & Taxes', 'Bill Amount', 'id', 'Map', 'Freight', 'Fuel',
       'Residential', 'Surcharge', 'Freight Adj', 'Fuel Adj', 'Surcharge Adj',
       'Duties/Tax'],
      dtype='object')

In [185]:
invoice_Flatten.shape

(6521, 41)

In [186]:
invoice_Flatten.isna().sum()

Invoice                       0
Invoice Date                  0
Order ID                      0
Order Number                  0
Profile                       0
Carrier                       0
Transaction Date              0
Tracking Number               0
Service Type                  0
Charge                        0
Package Quantity           3254
Packaging Type             3456
Entered Weight (LB)           0
Billed Weight (LB)            0
Dim Length                  204
Dim Width                   204
Dim Height                  204
Zone                          0
Origin State                  0
Origin Country                0
Receiver Name              5269
Receiver Company           1152
Receiver Address Line 1       0
Receiver Address Line 2    5999
Receiver City                 0
Receiver State               11
Receiver Zip                  0
Receiver Country              0
Customs Value              3456
Duty & Taxes                  0
Bill Amount                   0
id      

In [187]:
#invoice_Flatten = invoice_Flatten[invoice_Flatten[['Dim Length', 'Dim Width', 'Dim Height']].notnull().any(axis=1)]
invoice_Flatten = invoice_Flatten[invoice_Flatten[['Dim Length', 'Dim Width', 'Dim Height']].notnull().any(axis=1)]

In [188]:
invoice_Flatten.shape

(6317, 41)

In [189]:
grouped_df = invoice_Flatten.groupby(
    ['Order Number', 'Tracking Number', 'Origin State', 'Transaction Date', 'Receiver Zip', 'Carrier', 'Service Type'],
    as_index=False
).agg({
    'Freight': 'sum',
    'Fuel': 'sum',
    'Residential': 'sum',
    'Surcharge': 'sum',
    'Freight Adj': 'sum',
    'Fuel Adj': 'sum',
    'Surcharge Adj': 'sum',
    'Duties/Tax': 'sum',
    'Entered Weight (LB)': 'first',
    'Billed Weight (LB)': 'first'
})

In [190]:
grouped_df.shape

(2808, 17)

In [ ]:
grouped_df2 = invoice_Flatten.groupby(
    ['Order Number', 'Origin State', 'Transaction Date', 'Receiver Zip', 'Carrier', 'Service Type'],
    as_index=False
).agg({
    'Freight': 'sum',
    'Fuel': 'sum',
    'Residential': 'sum',
    'Surcharge': 'sum',
    'Freight Adj': 'sum',
    'Fuel Adj': 'sum',
    'Surcharge Adj': 'sum',
    'Duties/Tax': 'sum',
    'Entered Weight (LB)': 'first',
    'Billed Weight (LB)': 'first'
})

In [ ]:
2808 is any null